### From a URL create a Dataframe

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
from tqdm.notebook import tqdm
import random

In [2]:
url = "https://www.billboard.com/charts/hot-100/"

In [3]:
top100 = requests.get(url)

In [4]:
top100.status_code

200

### Search for the songs in the URL

In [5]:
soup=BeautifulSoup(top100.content,'html.parser')

In [6]:
soup.select('h3.c-title.a-no-trucate')[0].text

'\nEasy On Me\n'

In [7]:
top100=len(soup.select('h3.c-title.a-no-trucate'))

In [8]:
top100

100

In [9]:
index = soup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)

In [10]:
index

'Easy On Me'

### Search for the artists in the URL

In [11]:
soup.select('span.c-label.a-no-trucate')[0].get_text(strip=True)

'Adele'

In [12]:
soup.select('span.c-label.a-no-trucate')[99].get_text(strip=True)

'The Weeknd'

### Create a DataFrame

In [13]:
title = []
artist = []

In [14]:
for i in tqdm(range(top100)):
    title.append(soup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
    artist.append(soup.select('span.c-label.a-no-trucate')[i].get_text(strip=True))

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
songs_top100=pd.DataFrame({'Title':title,'Artist':artist})

In [16]:
songs_top100

,Title,Artist
0,Easy On Me,Adele
1,Stay,The Kid LAROI & Justin Bieber
2,Industry Baby,Lil Nas X & Jack Harlow
3,All Too Well (Taylor's Version),Taylor Swift
4,Oh My God,Adele
...,...,...
95,Too Easy,Gunna & Future
96,Escape Plan,Travis Scott
97,Who's In Your Head,Jonas Brothers
98,Jugaste y Sufri,Eslabon Armado Featuring DannyLux


In [17]:
songs_top100[92:]

,Title,Artist
92,Volvi,Aventura x Bad Bunny
93,Maybach,42 Dugg Featuring Future
94,Baddest,"Yung Bleu, Chris Brown & 2 Chainz"
95,Too Easy,Gunna & Future
96,Escape Plan,Travis Scott
97,Who's In Your Head,Jonas Brothers
98,Jugaste y Sufri,Eslabon Armado Featuring DannyLux
99,Take My Breath,The Weeknd


### Create a prototype where:
- The user can ask for one song and see if its in the playlist Top hot 100 songs
- Returns a new song if the first one is in the playlist

In [18]:
song= input("Give us a song name to get a recommendation!: ")

Give us a song name to get a recommendation!: stay


In [19]:
check = songs_top100[songs_top100['Title'].str.lower().str.replace(" "," ").str.contains(song)]

In [20]:
index = check.index.tolist()

In [21]:
def recommendation_engine():
    recommendation_engine()

In [22]:
song= input("Give us a song name to get a recommendation!: ")

check = songs_top100[songs_top100['Title'].str.lower().str.replace(" "," ").str.contains(song)]
index = check.index.tolist()

if len(index) == 0:
    print("This song is not on the Top 100 hot playlist! ")
else:
    answer = input("Do you mean " + songs_top100.Title[index].values[0] + " by " + songs_top100.Artist[index].values[0] + "?")

    if answer.lower() == 'yes':
        suggestion = songs_top100.sample().index.tolist()
        print("Nice! This is a hot song! You might also like " + songs_top100['Title'][suggestion].item() + " by " + songs_top100['Artist'][suggestion].item())

    else:
        print("that song is not in the playlist, try another song!")
    
    

Give us a song name to get a recommendation!: stay
Do you mean Stay by The Kid LAROI & Justin Bieber?yes
Nice! This is a hot song! You might also like Enemy by Imagine Dragons X JID


# Import Spotify

In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### Credentials

In [24]:
import getpass

client_id=str(getpass.getpass('client_id?'))
client_secret=str(getpass.getpass('client_secret?'))

client_id?········
client_secret?········


In [25]:
# embedding our getpass credentials in the access key

sp= spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

## Import playlists by genre
- I will first import a POP playlist

In [26]:
## https://open.spotify.com/playlist/37i9dQZF1DX1ngEVM0lKrb?si=332a03edd1db4135
## 37i9dQZF1DX1ngEVM0lKrb

In [27]:
pop_playlist = sp.user_playlist_tracks("Sarah Eringaard", "1HWoagBCUTlqigP88enL5r")

In [28]:
pop_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [29]:
len(pop_playlist['items'])

100

In [30]:
def get_playlist_tracks(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [31]:
results2=get_playlist_tracks("Sarah Eringaard", "1HWoagBCUTlqigP88enL5r")

In [32]:
len(results2)

473

In [33]:
results2[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [34]:
results2[0]['track']["uri"].rsplit(':',1)

['spotify:track', '04aAxqtGp5pv12UXAg4pkq']

In [35]:
results2[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [36]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results2

In [37]:
for r in results2:
    song_URI.append(r["track"]["uri"])

In [38]:
for r in results2: 
    song_ID.append(r["track"]["id"])

In [39]:
for r in results2: 
    song_name.append(r["track"]["name"])

In [40]:
for r in results2: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [41]:
import pandas as pd

In [42]:
playlist_pop_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})


In [43]:
playlist_pop_info

,song_name,song_ID,song_artist
0,Centuries,04aAxqtGp5pv12UXAg4pkq,Fall Out Boy
1,Counting Stars,6sy3LkhNFjJWlaeSMNwQ62,OneRepublic
2,Animals,16pwlVsypm4aDKMhXdOuXg,Maroon 5
3,Take Me to Church,0aOluBqXYd0rFSCsgDyAWX,Hozier
4,Rude,6RtPijgfPKROxEzTHNRiDp,MAGIC!
...,...,...,...
468,Jenny,2rpVmxAGGc18wpBREaoE0R,Studio Killers
469,I'm Gonna Be (500 Miles),66S14BkJDxgkYxLl5DCqOz,The Proclaimers
470,My Humps,19JdbdEwQI7bCxefMokZO8,Black Eyed Peas
471,I Just Wanna Run,3WLLJzllBEgPiSA0qd3BN3,The Downtown Fiction


### Add a new column for genre in playlist_pop_info

In [44]:
playlist_pop_info.insert(3, 'Genre', 'Pop') 

In [45]:
playlist_pop_info

,song_name,song_ID,song_artist,Genre
0,Centuries,04aAxqtGp5pv12UXAg4pkq,Fall Out Boy,Pop
1,Counting Stars,6sy3LkhNFjJWlaeSMNwQ62,OneRepublic,Pop
2,Animals,16pwlVsypm4aDKMhXdOuXg,Maroon 5,Pop
3,Take Me to Church,0aOluBqXYd0rFSCsgDyAWX,Hozier,Pop
4,Rude,6RtPijgfPKROxEzTHNRiDp,MAGIC!,Pop
...,...,...,...,...
468,Jenny,2rpVmxAGGc18wpBREaoE0R,Studio Killers,Pop
469,I'm Gonna Be (500 Miles),66S14BkJDxgkYxLl5DCqOz,The Proclaimers,Pop
470,My Humps,19JdbdEwQI7bCxefMokZO8,Black Eyed Peas,Pop
471,I Just Wanna Run,3WLLJzllBEgPiSA0qd3BN3,The Downtown Fiction,Pop


## Import playlists by genre
- I will import now a INDIE playlist

In [46]:
## https://open.spotify.com/playlist/7lnCgcTxLTTcOqvgoS80sC?si=923d97089cf041ea
## oceanify

In [47]:
indie_playlist = sp.user_playlist_tracks("oceanify", "7lnCgcTxLTTcOqvgoS80sC")

In [48]:
indie_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [49]:
len(indie_playlist['items'])

100

In [50]:
def get_playlist_tracks1(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [51]:
results3=get_playlist_tracks1("oceanify", "7lnCgcTxLTTcOqvgoS80sC")

In [52]:
len(results3)

170

In [53]:
results3[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [54]:
results3[0]['track']["uri"].rsplit(':',1)

['spotify:track', '3Wrjm47oTz2sjIgck11l5e']

In [55]:
results3[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [56]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results3

In [57]:
for r in results3:
    song_URI.append(r["track"]["uri"])
    
for r in results3: 
    song_ID.append(r["track"]["id"])
    
for r in results3: 
    song_name.append(r["track"]["name"])
    
for r in results3: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [58]:
playlist_indie_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})


In [59]:
playlist_indie_info

,song_name,song_ID,song_artist
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure
4,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby
...,...,...,...
165,Maybe - Extended Version,5M9FFdvLfKPkORIRERHzKY,Elizabeth Cuite
166,Deja Vu,5PWT6FhFPn2E5dv5tUuqNE,Luvlee
167,One in a Million,0B2gr2o765XiWU6dGKmktU,Will Whisson
168,Comin’ Home (Stewart Sullivan Mix),49FulGQwWUrWVVB69TfHAr,Dominoe


### Add a new column for genre in playlist_indie_info

In [60]:
playlist_indie_info.insert(3, 'Genre', 'Indie') 

In [61]:
playlist_indie_info

,song_name,song_ID,song_artist,Genre
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie
4,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby,Indie
...,...,...,...,...
165,Maybe - Extended Version,5M9FFdvLfKPkORIRERHzKY,Elizabeth Cuite,Indie
166,Deja Vu,5PWT6FhFPn2E5dv5tUuqNE,Luvlee,Indie
167,One in a Million,0B2gr2o765XiWU6dGKmktU,Will Whisson,Indie
168,Comin’ Home (Stewart Sullivan Mix),49FulGQwWUrWVVB69TfHAr,Dominoe,Indie


## Import playlists by genre
- I will import now a ROCK playlist

In [62]:
## https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=6870797da01948f7
## Spotify

In [63]:
rock_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DWXRqgorJj26U")

In [64]:
rock_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [65]:
len(rock_playlist['items'])

100

In [66]:
def get_playlist_tracks2(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [67]:
results4=get_playlist_tracks2("Spotify", "37i9dQZF1DWXRqgorJj26U")

In [68]:
len(results4)

166

In [69]:
results4[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [70]:
results4[0]['track']["uri"].rsplit(':',1)

['spotify:track', '57JVGBtBLCfHw2muk5416J']

In [71]:
results4[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [72]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results4

In [73]:
for r in results4:
    song_URI.append(r["track"]["uri"])
    
for r in results4: 
    song_ID.append(r["track"]["id"])
    
for r in results4: 
    song_name.append(r["track"]["name"])
    
for r in results4: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [74]:
playlist_rock_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [75]:
playlist_rock_info

,song_name,song_ID,song_artist
0,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,Queen
1,Paradise City,3YBZIN3rekqsKxbJc9FZko,Guns N' Roses
2,Black Dog - Remaster,3qT4bUD1MaWpGrTwcvguhb,Led Zeppelin
3,Highway to Hell,2zYzyRzz6pRmhPzyfMEC8s,AC/DC
4,Dream On,5MxNLUsfh7uzROypsoO5qe,Aerosmith
...,...,...,...
161,Ramble On - 1990 Remaster,3MODES4TNtygekLl146Dxd,Led Zeppelin
162,Crazy On You,5zH710lFSLtkHbMkslLDjR,Heart
163,Foreplay / Long Time,5UYPZZDQfpCE7A0FDDqv4k,Boston
164,My Generation - Mono Version,45s88Xopo6KvHc0PQ05aGg,The Who


### Add a new column for genre in playlist_rock_info

In [76]:
playlist_rock_info.insert(3, 'Genre', 'Rock') 

In [77]:
playlist_rock_info

,song_name,song_ID,song_artist,Genre
0,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,Queen,Rock
1,Paradise City,3YBZIN3rekqsKxbJc9FZko,Guns N' Roses,Rock
2,Black Dog - Remaster,3qT4bUD1MaWpGrTwcvguhb,Led Zeppelin,Rock
3,Highway to Hell,2zYzyRzz6pRmhPzyfMEC8s,AC/DC,Rock
4,Dream On,5MxNLUsfh7uzROypsoO5qe,Aerosmith,Rock
...,...,...,...,...
161,Ramble On - 1990 Remaster,3MODES4TNtygekLl146Dxd,Led Zeppelin,Rock
162,Crazy On You,5zH710lFSLtkHbMkslLDjR,Heart,Rock
163,Foreplay / Long Time,5UYPZZDQfpCE7A0FDDqv4k,Boston,Rock
164,My Generation - Mono Version,45s88Xopo6KvHc0PQ05aGg,The Who,Rock


## Import playlists by genre
- I will import now a TROPICAL HOUSE playlist

In [78]:
## https://open.spotify.com/playlist/37i9dQZF1DX0AMssoUKCz7?si=8509345b0f7b4117
## Spotify

In [79]:
tropical_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX0AMssoUKCz7")

In [80]:
tropical_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [81]:
len(tropical_playlist['items'])

100

In [82]:
def get_playlist_tracks3(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [83]:
results5=get_playlist_tracks3("Spotify", "37i9dQZF1DX0AMssoUKCz7")

In [84]:
len(results5)

200

In [85]:
results5[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [86]:
results5[0]['track']["uri"].rsplit(':',1)

['spotify:track', '1c1sdxrYLIiuJOlE7PPttb']

In [87]:
results5[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [88]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results5

In [89]:
for r in results5:
    song_URI.append(r["track"]["uri"])
    
for r in results5: 
    song_ID.append(r["track"]["id"])
    
for r in results5: 
    song_name.append(r["track"]["name"])
    
for r in results5: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [90]:
playlist_tropical_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [91]:
playlist_tropical_info

,song_name,song_ID,song_artist
0,Love Me Now (feat. Zoe Wees),1c1sdxrYLIiuJOlE7PPttb,Kygo
1,Skinny Dip (Komodo) [feat. Philip Strand],79m1I109FyZMx3VwQZAEQM,Deepend
2,Summer Feeling,793OuR8rJ0zpVp708ONAxO,Matoma
3,Ride,4C020pCjavabKAGmjZZRmh,Madism
4,Call On Me (feat. Georgia Ku),3pBv3LfvfPft2TqHleqKHo,Sam Feldt
...,...,...,...
195,Recognise,3YwyAnL9NlqAK9V833qiGz,Lost Frequencies
196,Ragga - Radio Edit,6JKSgnPypPIs08k1SGOhXD,Bolier
197,The Last Ones Standing,4X6wMQec2NHdBhEGUwqUFV,Tep No
198,Wishing Well,3a4UuAErF1VMAkwqzkc1tj,Sam Feldt


### Add a new column for genre in playlist_tropical_info

In [92]:
playlist_tropical_info.insert(3, 'Genre', 'Tropical') 

In [93]:
playlist_tropical_info

,song_name,song_ID,song_artist,Genre
0,Love Me Now (feat. Zoe Wees),1c1sdxrYLIiuJOlE7PPttb,Kygo,Tropical
1,Skinny Dip (Komodo) [feat. Philip Strand],79m1I109FyZMx3VwQZAEQM,Deepend,Tropical
2,Summer Feeling,793OuR8rJ0zpVp708ONAxO,Matoma,Tropical
3,Ride,4C020pCjavabKAGmjZZRmh,Madism,Tropical
4,Call On Me (feat. Georgia Ku),3pBv3LfvfPft2TqHleqKHo,Sam Feldt,Tropical
...,...,...,...,...
195,Recognise,3YwyAnL9NlqAK9V833qiGz,Lost Frequencies,Tropical
196,Ragga - Radio Edit,6JKSgnPypPIs08k1SGOhXD,Bolier,Tropical
197,The Last Ones Standing,4X6wMQec2NHdBhEGUwqUFV,Tep No,Tropical
198,Wishing Well,3a4UuAErF1VMAkwqzkc1tj,Sam Feldt,Tropical


## Import playlists by genre
- I will import now a HIPHOP playlist

In [94]:
## https://open.spotify.com/playlist/33I6RpefRQcRh69xEczaKT?si=8bc85c31fb894e45
## Leon Bouw

In [95]:
hiphop_playlist = sp.user_playlist_tracks("Leon Bouw", "33I6RpefRQcRh69xEczaKT")

In [96]:
hiphop_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [97]:
len(hiphop_playlist['items'])

100

In [98]:
def get_playlist_tracks4(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [99]:
results6=get_playlist_tracks3("Leon Bouw", "33I6RpefRQcRh69xEczaKT")

In [100]:
len(results6)

265

In [101]:
results6[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [102]:
results6[0]['track']["uri"].rsplit(':',1)

['spotify:track', '1YrHyDbH17btvI2rFTkbCQ']

In [103]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results6

In [104]:
for r in results6:
    song_URI.append(r["track"]["uri"])
    
for r in results6: 
    song_ID.append(r["track"]["id"])
    
for r in results6: 
    song_name.append(r["track"]["name"])
    
for r in results6: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [105]:
playlist_hiphop_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [106]:
playlist_hiphop_info

,song_name,song_ID,song_artist
0,Bitch Please,1YrHyDbH17btvI2rFTkbCQ,Snoop Dogg
1,How We Do,6mZcNu9i8uhOYMiEAyEx81,The Game
2,No Hands (feat. Roscoe Dash & Wale),6GgPsuz0HEO0nrO2T0QhDv,Waka Flocka Flame
3,Gorgeous,1yhQGEykn7ZTNLwcIjOAwZ,Kanye West
4,99 Problems,4HsL5A46wa0O2jQ0ngj3H7,JAY-Z
...,...,...,...
260,Yo Sassy Ways,0J1siBLQBHYgYKeFgu7RAY,Warren G
261,Cisco Kid,0CJ3Y3V6UQn5X7IjXDO3IO,Redman
262,Woah! - Radio Mix,5fZjSIReYPJIPwpvrsyW3o,Black Rob
263,I Don't Wanna Care Right Now (feat. MDMA),5yhXLtQL9waNMjgkl8MsMP,Lupe Fiasco


### Add a new column for genre in playlist_hiphop_info

In [107]:
playlist_hiphop_info.insert(3, 'Genre', 'HipHop') 

In [108]:
playlist_hiphop_info

,song_name,song_ID,song_artist,Genre
0,Bitch Please,1YrHyDbH17btvI2rFTkbCQ,Snoop Dogg,HipHop
1,How We Do,6mZcNu9i8uhOYMiEAyEx81,The Game,HipHop
2,No Hands (feat. Roscoe Dash & Wale),6GgPsuz0HEO0nrO2T0QhDv,Waka Flocka Flame,HipHop
3,Gorgeous,1yhQGEykn7ZTNLwcIjOAwZ,Kanye West,HipHop
4,99 Problems,4HsL5A46wa0O2jQ0ngj3H7,JAY-Z,HipHop
...,...,...,...,...
260,Yo Sassy Ways,0J1siBLQBHYgYKeFgu7RAY,Warren G,HipHop
261,Cisco Kid,0CJ3Y3V6UQn5X7IjXDO3IO,Redman,HipHop
262,Woah! - Radio Mix,5fZjSIReYPJIPwpvrsyW3o,Black Rob,HipHop
263,I Don't Wanna Care Right Now (feat. MDMA),5yhXLtQL9waNMjgkl8MsMP,Lupe Fiasco,HipHop


## Import playlists by genre
- I will import now a LATIN playlist

In [109]:
## https://open.spotify.com/playlist/4DLmZ0UM8TH0UXRfI0Mvu6?si=138328e1e1b44091
## MiKeL

In [110]:
latin_playlist = sp.user_playlist_tracks("MiKeL", "4DLmZ0UM8TH0UXRfI0Mvu6")

In [111]:
latin_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [112]:
len(latin_playlist['items'])

100

In [113]:
def get_playlist_tracks5(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [114]:
results7=get_playlist_tracks5("MiKeL", "4DLmZ0UM8TH0UXRfI0Mvu6")

In [115]:
len(results7)

269

In [116]:
results7[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [117]:
results7[0]['track']["uri"].rsplit(':',1)

['spotify:track', '1ZcrHpBbILPNfhBtPhQVHh']

In [118]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results7

In [119]:
for r in results7:
    song_URI.append(r["track"]["uri"])
    
for r in results7: 
    song_ID.append(r["track"]["id"])
    
for r in results7: 
    song_name.append(r["track"]["name"])
    
for r in results7: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [120]:
playlist_latin_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [121]:
playlist_latin_info

,song_name,song_ID,song_artist
0,Se Menea,1ZcrHpBbILPNfhBtPhQVHh,Don Omar
1,Una Vaina Loca,2qEoAz0i6yEz5dPggABcLH,Fuego
2,Señor Juez,2PH533J1xudlfZ5GaHAdVX,Ozuna
3,Ella - Remix,7iHHlCbsFou5DZSW5HCPfs,Boza
4,Unfollow,6GsD646HrIDJJpzoY9T1Rt,Duki
...,...,...,...
264,Ay Vamos,6Ges5C2IE738iJh4HyQizQ,J Balvin
265,Love Not War (The Tampa Beat),4Lt6GXGzYsa1tgkv3nGSTm,Jason Derulo
266,BICHOTA,7vrJn5hDSXRmdXoR30KgF1,KAROL G
267,Hey DJ,1XgpK29CGGjZnxPYkiRbh4,CNCO


### Add a new column for genre in playlist_latin_info

In [122]:
playlist_latin_info.insert(3, 'Genre', 'Latin') 

In [123]:
playlist_latin_info

,song_name,song_ID,song_artist,Genre
0,Se Menea,1ZcrHpBbILPNfhBtPhQVHh,Don Omar,Latin
1,Una Vaina Loca,2qEoAz0i6yEz5dPggABcLH,Fuego,Latin
2,Señor Juez,2PH533J1xudlfZ5GaHAdVX,Ozuna,Latin
3,Ella - Remix,7iHHlCbsFou5DZSW5HCPfs,Boza,Latin
4,Unfollow,6GsD646HrIDJJpzoY9T1Rt,Duki,Latin
...,...,...,...,...
264,Ay Vamos,6Ges5C2IE738iJh4HyQizQ,J Balvin,Latin
265,Love Not War (The Tampa Beat),4Lt6GXGzYsa1tgkv3nGSTm,Jason Derulo,Latin
266,BICHOTA,7vrJn5hDSXRmdXoR30KgF1,KAROL G,Latin
267,Hey DJ,1XgpK29CGGjZnxPYkiRbh4,CNCO,Latin


## Import playlists by genre
- I will import now a SPANISH playlist

In [124]:
## https://open.spotify.com/playlist/1oUJ31mriViDmfMliMXRRZ?si=c25b5ae7a9d546cb
## Gr Fausto

In [125]:
spanish_playlist = sp.user_playlist_tracks("Gr Fausto", "1oUJ31mriViDmfMliMXRRZ")

In [126]:
spanish_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [127]:
len(spanish_playlist['items'])

100

In [128]:
def get_playlist_tracks6(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [129]:
results8=get_playlist_tracks6("Gr Fausto", "1oUJ31mriViDmfMliMXRRZ")

In [130]:
len(results8)

597

In [131]:
results8[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [132]:
results8[0]['track']["uri"].rsplit(':',1)

['spotify:track', '3v1dCP3hk2djfWryqfp7sx']

In [133]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results8

In [134]:
for r in results8:
    song_URI.append(r["track"]["uri"])
    
for r in results8: 
    song_ID.append(r["track"]["id"])
    
for r in results8: 
    song_name.append(r["track"]["name"])
    
for r in results8: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [135]:
playlist_spanish_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [136]:
playlist_spanish_info

,song_name,song_ID,song_artist
0,Caminando por la vida,3v1dCP3hk2djfWryqfp7sx,Melendi
1,Soldadito marinero,0eVborSuxUeSg0meWYd9dZ,Fito y Fitipaldis
2,Walter Palmeras,58UxBSr2y1b7gun2Vjt8dg,Taburete
3,Caminito A Motel,3VptrSkImJN0c5KCEBHE65,Taburete
4,Amos del Piano Bar,5wnIjet6vh2BaIX2gl4Mgz,Taburete
...,...,...,...
592,Cuando Brille el Sol (with Manuel Espana),2T0hFOkypPat3w4K5mc7fc,Melocos
593,Yo no te pido la luna,1hI1st8dvPmr1S55XDkavG,Sergio Dalma
594,A Mi Manera,5yY2UPVQ9EBKBXh2dGtU6U,Siempre Asi
595,Sopa fría,1zLJEnxuZBgPbwMrDMiZAf,M-Clan


### Add a new column for genre in playlist_spanish_info

In [137]:
playlist_spanish_info.insert(3, 'Genre', 'Spanish') 

In [138]:
playlist_spanish_info

,song_name,song_ID,song_artist,Genre
0,Caminando por la vida,3v1dCP3hk2djfWryqfp7sx,Melendi,Spanish
1,Soldadito marinero,0eVborSuxUeSg0meWYd9dZ,Fito y Fitipaldis,Spanish
2,Walter Palmeras,58UxBSr2y1b7gun2Vjt8dg,Taburete,Spanish
3,Caminito A Motel,3VptrSkImJN0c5KCEBHE65,Taburete,Spanish
4,Amos del Piano Bar,5wnIjet6vh2BaIX2gl4Mgz,Taburete,Spanish
...,...,...,...,...
592,Cuando Brille el Sol (with Manuel Espana),2T0hFOkypPat3w4K5mc7fc,Melocos,Spanish
593,Yo no te pido la luna,1hI1st8dvPmr1S55XDkavG,Sergio Dalma,Spanish
594,A Mi Manera,5yY2UPVQ9EBKBXh2dGtU6U,Siempre Asi,Spanish
595,Sopa fría,1zLJEnxuZBgPbwMrDMiZAf,M-Clan,Spanish


## Import playlists by genre
- I will import now a JAZZ playlist

In [139]:
## https://open.spotify.com/playlist/37i9dQZF1DX0SM0LYsmbMT?si=a455c71c0ea24707
## Spotify

In [140]:
jazz_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX0SM0LYsmbMT")

In [141]:
jazz_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [142]:
len(jazz_playlist['items'])

100

In [143]:
def get_playlist_tracks7(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [144]:
results9=get_playlist_tracks7("Spotify", "37i9dQZF1DX0SM0LYsmbMT")

In [145]:
len(results9)

300

In [146]:
results9[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [147]:
results9[0]['track']["uri"].rsplit(':',1)

['spotify:track', '68XSHyQQlIhs6ytuHIr6Uo']

In [148]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results9

In [149]:
for r in results9:
    song_URI.append(r["track"]["uri"])
    
for r in results9: 
    song_ID.append(r["track"]["id"])
    
for r in results9: 
    song_name.append(r["track"]["name"])
    
for r in results9: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [150]:
playlist_jazz_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [151]:
playlist_jazz_info

,song_name,song_ID,song_artist
0,Memories,68XSHyQQlIhs6ytuHIr6Uo,Mr. Käfer
1,Above the Clouds,0ogIUn5t00tseKzRngQfnu,Oilix
2,Sad Emoji,1biA4kEJ8M2tPrWfwy9uID,Katashi Kaitō
3,Bubbles,3ocMyz1hZX5pl87L4xgadr,Glimlip
4,Soulsearching,7bKVWzUXNgcEdi32sECdD1,Hoffy Beats
...,...,...,...
295,Rooftop,1t8wgBJKb84ig6bZ84H041,Cookin Soul
296,Lilutu,2qHy5286NNXTEMmXWOQ9q5,Sátyr
297,It's Sunny,18CTxtDklwHVJ3C9neDUEN,ommood
298,ST13,43got7b21DC6igGK9P2oHu,Jazzy James


### Add a new column for genre in playlist_spanish_info

In [152]:
playlist_jazz_info.insert(3, 'Genre', 'Jazz') 

In [153]:
playlist_jazz_info

,song_name,song_ID,song_artist,Genre
0,Memories,68XSHyQQlIhs6ytuHIr6Uo,Mr. Käfer,Jazz
1,Above the Clouds,0ogIUn5t00tseKzRngQfnu,Oilix,Jazz
2,Sad Emoji,1biA4kEJ8M2tPrWfwy9uID,Katashi Kaitō,Jazz
3,Bubbles,3ocMyz1hZX5pl87L4xgadr,Glimlip,Jazz
4,Soulsearching,7bKVWzUXNgcEdi32sECdD1,Hoffy Beats,Jazz
...,...,...,...,...
295,Rooftop,1t8wgBJKb84ig6bZ84H041,Cookin Soul,Jazz
296,Lilutu,2qHy5286NNXTEMmXWOQ9q5,Sátyr,Jazz
297,It's Sunny,18CTxtDklwHVJ3C9neDUEN,ommood,Jazz
298,ST13,43got7b21DC6igGK9P2oHu,Jazzy James,Jazz


## Import playlists by genre
- I will import now a Deep House playlist

In [154]:
## https://open.spotify.com/playlist/37i9dQZF1DX2TRYkJECvfC?si=e54832a9a20e4451
## Spotify

In [155]:
deep_house_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX2TRYkJECvfC")

In [156]:
deep_house_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [157]:
len(deep_house_playlist['items'])

100

In [158]:
def get_playlist_tracks8(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [159]:
results10=get_playlist_tracks8("Spotify", "37i9dQZF1DX2TRYkJECvfC")

In [160]:
len(results10)

200

In [161]:
results10[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [162]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results10

In [163]:
for r in results10:
    song_URI.append(r["track"]["uri"])
    
for r in results10: 
    song_ID.append(r["track"]["id"])
    
for r in results10: 
    song_name.append(r["track"]["name"])
    
for r in results10: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [164]:
playlist_deep_house_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [165]:
playlist_deep_house_info

,song_name,song_ID,song_artist
0,Boats,5NOroGtZSBsdCid7axlyXj,Xinobi
1,Designer Love - Short Version,5NfgFRdfNQ4HFVPGRoVMU6,MARIA Die RUHE
2,Erase,1BbXlbs8pWeYI2yGpGhuzA,Ben Böhmer
3,Renaissance,1mqPLbDc8DCRlTuyF26ste,Honeydripper
4,Broken Parts,1TS3hIyeGdWqeBQyj5HgoT,Eli & Fur
...,...,...,...
195,Sofia's Theme,16qgL8moRXVRZA5K0yU4yZ,TÂCHES
196,Loops For Healing,1RL9gghbn6lqDyCTPteNfB,Octo Octa
197,What You Want,1RuMPMUxwdAZ0tMcaUwQiw,glue70
198,Spectrum - Original,3XHORygIaZ3k7EKNnbZEUV,Collective Machine


### Add a new column for genre in playlist_deep_house_info

In [166]:
playlist_deep_house_info.insert(3, 'Genre', 'Deep House') 

In [167]:
playlist_deep_house_info

,song_name,song_ID,song_artist,Genre
0,Boats,5NOroGtZSBsdCid7axlyXj,Xinobi,Deep House
1,Designer Love - Short Version,5NfgFRdfNQ4HFVPGRoVMU6,MARIA Die RUHE,Deep House
2,Erase,1BbXlbs8pWeYI2yGpGhuzA,Ben Böhmer,Deep House
3,Renaissance,1mqPLbDc8DCRlTuyF26ste,Honeydripper,Deep House
4,Broken Parts,1TS3hIyeGdWqeBQyj5HgoT,Eli & Fur,Deep House
...,...,...,...,...
195,Sofia's Theme,16qgL8moRXVRZA5K0yU4yZ,TÂCHES,Deep House
196,Loops For Healing,1RL9gghbn6lqDyCTPteNfB,Octo Octa,Deep House
197,What You Want,1RuMPMUxwdAZ0tMcaUwQiw,glue70,Deep House
198,Spectrum - Original,3XHORygIaZ3k7EKNnbZEUV,Collective Machine,Deep House


## Import playlists by genre
- I will import now a Disney playlist

In [168]:
## https://open.spotify.com/playlist/37i9dQZF1DX8C9xQcOrE6T?si=d2057893dbca4cb2
## Spotify

In [169]:
disney_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX8C9xQcOrE6T")

In [170]:
disney_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [171]:
len(disney_playlist['items'])

100

In [172]:
def get_playlist_tracks9(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [173]:
results11=get_playlist_tracks9("Spotify", "37i9dQZF1DX8C9xQcOrE6T")

In [174]:
len(results11)

102

In [175]:
results11[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [176]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results11

In [177]:
for r in results11:
    song_URI.append(r["track"]["uri"])
    
for r in results11: 
    song_ID.append(r["track"]["id"])
    
for r in results11: 
    song_name.append(r["track"]["name"])
    
for r in results11: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [178]:
playlist_disney_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [179]:
playlist_disney_info

,song_name,song_ID,song_artist
0,Into the Unknown,3Z0oQ8r78OUaHvGPiDBR3W,Idina Menzel
1,How Far I'll Go,6mb6lVLNrcUgLnEN8QnDJd,Auli'i Cravalho
2,"Colombia, Mi Encanto",2Q5q1q01KOdHt5fM00WUJ8,Carlos Vives
3,Un Poco Loco,0OFknyqxmSQ42SoKxWVTok,Anthony Gonzalez
4,The Family Madrigal,4b1yxSdlumA8N4fEk4UOZp,Stephanie Beatriz
...,...,...,...
97,"When Can I See You Again? - From ""Wreck-It Ral...",5ZkAx8zjLiSs1nMmBwJoZS,Owl City
98,Zero,4Bi2qsF5AWin93USPh5fQK,Imagine Dragons
99,Spirit,38SueFAjT6jtFneyoQamYi,Beyoncé
100,In This Place,2cOE7d35PyfAh9M7DglPk0,Julia Michaels


### Add a new column for genre in playlist_disney_info

In [180]:
playlist_disney_info.insert(3, 'Genre', 'Disney') 

In [181]:
playlist_disney_info

,song_name,song_ID,song_artist,Genre
0,Into the Unknown,3Z0oQ8r78OUaHvGPiDBR3W,Idina Menzel,Disney
1,How Far I'll Go,6mb6lVLNrcUgLnEN8QnDJd,Auli'i Cravalho,Disney
2,"Colombia, Mi Encanto",2Q5q1q01KOdHt5fM00WUJ8,Carlos Vives,Disney
3,Un Poco Loco,0OFknyqxmSQ42SoKxWVTok,Anthony Gonzalez,Disney
4,The Family Madrigal,4b1yxSdlumA8N4fEk4UOZp,Stephanie Beatriz,Disney
...,...,...,...,...
97,"When Can I See You Again? - From ""Wreck-It Ral...",5ZkAx8zjLiSs1nMmBwJoZS,Owl City,Disney
98,Zero,4Bi2qsF5AWin93USPh5fQK,Imagine Dragons,Disney
99,Spirit,38SueFAjT6jtFneyoQamYi,Beyoncé,Disney
100,In This Place,2cOE7d35PyfAh9M7DglPk0,Julia Michaels,Disney


## Import playlists by genre
- I will import now a Techno playlist

In [182]:
## https://open.spotify.com/playlist/1ttMvmcjWe9JKZybeYjtnU?si=15ee0122f19449ee
## Tunefullymusic

In [183]:
techno_playlist = sp.user_playlist_tracks("Tunefullymusic", "1ttMvmcjWe9JKZybeYjtnU")

In [184]:
techno_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [185]:
len(techno_playlist['items'])

100

In [186]:
def get_playlist_tracks10(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [187]:
results12=get_playlist_tracks10("Spotify", "1ttMvmcjWe9JKZybeYjtnU")

In [188]:
len(results12)

1021

In [189]:
results12[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [190]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results12

In [191]:
for r in results12:
    song_URI.append(r["track"]["uri"])
    
for r in results12: 
    song_ID.append(r["track"]["id"])
    
for r in results12: 
    song_name.append(r["track"]["name"])
    
for r in results12: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [192]:
playlist_techno_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [193]:
playlist_techno_info

,song_name,song_ID,song_artist
0,Gimme Some,6170lJSBtiM5trzOtHtdD2,Weval
1,To Let Myself Go,4IQgbTjXZfINWOS5TKL6T0,The Avener
2,In the Morning,01wpPOPqQ3XyS6hBN00HfK,ZHU
3,Love Me - 4ЯR & Cutneck Remix,7wBsua0Fr50vOEHzABrlsc,Damian Spider
4,West Coast,5Y6nVaayzitvsD5F7nr3DV,Lana Del Rey
...,...,...,...
1016,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn
1017,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation
1018,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea
1019,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage


### Add a new column for genre in playlist_disney_info

In [194]:
playlist_techno_info.insert(3, 'Genre', 'Techno') 

In [195]:
playlist_techno_info

,song_name,song_ID,song_artist,Genre
0,Gimme Some,6170lJSBtiM5trzOtHtdD2,Weval,Techno
1,To Let Myself Go,4IQgbTjXZfINWOS5TKL6T0,The Avener,Techno
2,In the Morning,01wpPOPqQ3XyS6hBN00HfK,ZHU,Techno
3,Love Me - 4ЯR & Cutneck Remix,7wBsua0Fr50vOEHzABrlsc,Damian Spider,Techno
4,West Coast,5Y6nVaayzitvsD5F7nr3DV,Lana Del Rey,Techno
...,...,...,...,...
1016,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn,Techno
1017,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation,Techno
1018,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea,Techno
1019,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage,Techno


## Joining all the dataframes into 1

In [196]:
frames=[playlist_indie_info, playlist_pop_info, playlist_rock_info, 
        playlist_tropical_info, playlist_hiphop_info
       ,playlist_latin_info,playlist_spanish_info,
       playlist_jazz_info, playlist_deep_house_info,
       playlist_disney_info, playlist_techno_info,
       ]
big_df=pd.concat(frames, axis=0)

In [197]:
big_df

,song_name,song_ID,song_artist,Genre
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie
4,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby,Indie
...,...,...,...,...
1016,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn,Techno
1017,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation,Techno
1018,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea,Techno
1019,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage,Techno


In [198]:
## reseting the index so the rows appear correctly

big_df.reset_index(drop=True, inplace=True)

In [199]:
big_df

,song_name,song_ID,song_artist,Genre
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie
4,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby,Indie
...,...,...,...,...
3758,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn,Techno
3759,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation,Techno
3760,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea,Techno
3761,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage,Techno


In [200]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3763 entries, 0 to 3762
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   song_name    3763 non-null   object
 1   song_ID      3763 non-null   object
 2   song_artist  3763 non-null   object
 3   Genre        3763 non-null   object
dtypes: object(4)
memory usage: 117.7+ KB


In [201]:
big_df['Genre'].dtypes

dtype('O')

# Audio features

In [202]:
from tqdm.notebook import tqdm

In [203]:
feats=[]

for song_ID in tqdm(range(len(big_df))):
    feats.extend(sp.audio_features(tracks=big_df['song_ID'][song_ID]))

  0%|          | 0/3763 [00:00<?, ?it/s]

In [204]:
feats

[{'danceability': 0.714,
  'energy': 0.8,
  'key': 11,
  'loudness': -4.808,
  'mode': 0,
  'speechiness': 0.0504,
  'acousticness': 0.127,
  'instrumentalness': 0,
  'liveness': 0.359,
  'valence': 0.589,
  'tempo': 134.002,
  'type': 'audio_features',
  'id': '3Wrjm47oTz2sjIgck11l5e',
  'uri': 'spotify:track:3Wrjm47oTz2sjIgck11l5e',
  'track_href': 'https://api.spotify.com/v1/tracks/3Wrjm47oTz2sjIgck11l5e',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Wrjm47oTz2sjIgck11l5e',
  'duration_ms': 211560,
  'time_signature': 4},
 {'danceability': 0.64,
  'energy': 0.755,
  'key': 1,
  'loudness': -4.077,
  'mode': 0,
  'speechiness': 0.0287,
  'acousticness': 0.0138,
  'instrumentalness': 0.0208,
  'liveness': 0.12,
  'valence': 0.744,
  'tempo': 116.883,
  'type': 'audio_features',
  'id': '4g3Ax56IslQkI6XVfYKVc5',
  'uri': 'spotify:track:4g3Ax56IslQkI6XVfYKVc5',
  'track_href': 'https://api.spotify.com/v1/tracks/4g3Ax56IslQkI6XVfYKVc5',
  'analysis_url': 'https://api.spo

In [205]:
feats[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [206]:
feats_df=pd.DataFrame(feats)

In [207]:
feats_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.714,0.800,11,-4.808,0,0.0504,0.12700,0.000000,0.3590,0.5890,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,0.640,0.755,1,-4.077,0,0.0287,0.01380,0.020800,0.1200,0.7440,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,0.355,0.918,1,-4.360,1,0.0746,0.00119,0.000000,0.0971,0.2400,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,0.462,0.836,11,-5.766,0,0.0621,0.00517,0.000000,0.1800,0.6710,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,0.808,0.381,1,-9.758,1,0.0584,0.34400,0.000003,0.1120,0.6870,102.052,audio_features,1H6A86dQ7BMdbxofxpQvcK,spotify:track:1H6A86dQ7BMdbxofxpQvcK,https://api.spotify.com/v1/tracks/1H6A86dQ7BMd...,https://api.spotify.com/v1/audio-analysis/1H6A...,174118,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,0.751,0.739,5,-8.557,0,0.0380,0.01450,0.884000,0.0902,0.0988,123.023,audio_features,3KaDPp6hfbnL4dAo57PNCI,spotify:track:3KaDPp6hfbnL4dAo57PNCI,https://api.spotify.com/v1/tracks/3KaDPp6hfbnL...,https://api.spotify.com/v1/audio-analysis/3KaD...,255630,4
3759,0.619,0.746,7,-10.770,1,0.0407,0.11600,0.880000,0.1370,0.1700,122.005,audio_features,4nSn1svDL1elP43MZuEc0K,spotify:track:4nSn1svDL1elP43MZuEc0K,https://api.spotify.com/v1/tracks/4nSn1svDL1el...,https://api.spotify.com/v1/audio-analysis/4nSn...,287213,4
3760,0.781,0.658,1,-10.312,1,0.0446,0.03060,0.742000,0.1120,0.1060,124.017,audio_features,4M5K6HYMIIoXLutK1v7g7U,spotify:track:4M5K6HYMIIoXLutK1v7g7U,https://api.spotify.com/v1/tracks/4M5K6HYMIIoX...,https://api.spotify.com/v1/audio-analysis/4M5K...,418065,4
3761,0.792,0.932,10,-6.781,1,0.0500,0.00117,0.916000,0.0723,0.3560,120.997,audio_features,0hUsx4d1Ur32PPQS9CX7yU,spotify:track:0hUsx4d1Ur32PPQS9CX7yU,https://api.spotify.com/v1/tracks/0hUsx4d1Ur32...,https://api.spotify.com/v1/audio-analysis/0hUs...,388696,4


### Merge the two dataframes so you have the songs and artists with de features

In [234]:
new_df=pd.concat([feats_df, all_df], axis=1)

In [231]:
new_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.714,0.800,11.0,-4.808,0.0,0.0504,0.12700,0.000000,0.3590,0.589,...,0.3590,0.5890,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,0.640,0.755,1.0,-4.077,0.0,0.0287,0.01380,0.020800,0.1200,0.744,...,0.1200,0.7440,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,0.355,0.918,1.0,-4.360,1.0,0.0746,0.00119,0.000000,0.0971,0.240,...,0.0971,0.2400,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,0.462,0.836,11.0,-5.766,0.0,0.0621,0.00517,0.000000,0.1800,0.671,...,0.1800,0.6710,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,0.808,0.381,1.0,-9.758,1.0,0.0584,0.34400,0.000003,0.1120,0.687,...,0.1120,0.6870,102.052,audio_features,1H6A86dQ7BMdbxofxpQvcK,spotify:track:1H6A86dQ7BMdbxofxpQvcK,https://api.spotify.com/v1/tracks/1H6A86dQ7BMd...,https://api.spotify.com/v1/audio-analysis/1H6A...,174118,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0902,0.0988,123.023,audio_features,3KaDPp6hfbnL4dAo57PNCI,spotify:track:3KaDPp6hfbnL4dAo57PNCI,https://api.spotify.com/v1/tracks/3KaDPp6hfbnL...,https://api.spotify.com/v1/audio-analysis/3KaD...,255630,4
3807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1370,0.1700,122.005,audio_features,4nSn1svDL1elP43MZuEc0K,spotify:track:4nSn1svDL1elP43MZuEc0K,https://api.spotify.com/v1/tracks/4nSn1svDL1el...,https://api.spotify.com/v1/audio-analysis/4nSn...,287213,4
3808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1120,0.1060,124.017,audio_features,4M5K6HYMIIoXLutK1v7g7U,spotify:track:4M5K6HYMIIoXLutK1v7g7U,https://api.spotify.com/v1/tracks/4M5K6HYMIIoX...,https://api.spotify.com/v1/audio-analysis/4M5K...,418065,4
3809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0723,0.3560,120.997,audio_features,0hUsx4d1Ur32PPQS9CX7yU,spotify:track:0hUsx4d1Ur32PPQS9CX7yU,https://api.spotify.com/v1/tracks/0hUsx4d1Ur32...,https://api.spotify.com/v1/audio-analysis/0hUs...,388696,4


In [232]:
new_df.isnull().sum()

danceability        48
energy              48
key                 48
loudness            48
mode                48
speechiness         48
acousticness        48
instrumentalness    48
liveness            48
valence             48
tempo               48
type                48
id                  48
uri                 48
track_href          48
analysis_url        48
duration_ms         48
time_signature      48
song_name            0
song_ID              0
song_artist          0
Genre                0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
type                 0
id                   0
uri                  0
track_href           0
analysis_url         0
duration_ms          0
time_signature       0
dtype: int64

In [235]:
new_df.dropna()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.714,0.800,11.0,-4.808,0.0,0.0504,0.12700,0.000000,0.3590,0.5890,...,0.3590,0.5890,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,0.640,0.755,1.0,-4.077,0.0,0.0287,0.01380,0.020800,0.1200,0.7440,...,0.1200,0.7440,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,0.355,0.918,1.0,-4.360,1.0,0.0746,0.00119,0.000000,0.0971,0.2400,...,0.0971,0.2400,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,0.462,0.836,11.0,-5.766,0.0,0.0621,0.00517,0.000000,0.1800,0.6710,...,0.1800,0.6710,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,0.808,0.381,1.0,-9.758,1.0,0.0584,0.34400,0.000003,0.1120,0.6870,...,0.1120,0.6870,102.052,audio_features,1H6A86dQ7BMdbxofxpQvcK,spotify:track:1H6A86dQ7BMdbxofxpQvcK,https://api.spotify.com/v1/tracks/1H6A86dQ7BMd...,https://api.spotify.com/v1/audio-analysis/1H6A...,174118,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,0.751,0.739,5.0,-8.557,0.0,0.0380,0.01450,0.884000,0.0902,0.0988,...,0.0698,0.2310,121.015,audio_features,1cWqKMY9UmPen1n6XH4byq,spotify:track:1cWqKMY9UmPen1n6XH4byq,https://api.spotify.com/v1/tracks/1cWqKMY9UmPe...,https://api.spotify.com/v1/audio-analysis/1cWq...,464132,4
3759,0.619,0.746,7.0,-10.770,1.0,0.0407,0.11600,0.880000,0.1370,0.1700,...,0.0967,0.4770,120.017,audio_features,2R2haDSmphwkn4kANdHU5P,spotify:track:2R2haDSmphwkn4kANdHU5P,https://api.spotify.com/v1/tracks/2R2haDSmphwk...,https://api.spotify.com/v1/audio-analysis/2R2h...,264046,4
3760,0.781,0.658,1.0,-10.312,1.0,0.0446,0.03060,0.742000,0.1120,0.1060,...,0.0566,0.0332,120.976,audio_features,7sSTTQbR0C0pKSlJYHWg0L,spotify:track:7sSTTQbR0C0pKSlJYHWg0L,https://api.spotify.com/v1/tracks/7sSTTQbR0C0p...,https://api.spotify.com/v1/audio-analysis/7sST...,237391,4
3761,0.792,0.932,10.0,-6.781,1.0,0.0500,0.00117,0.916000,0.0723,0.3560,...,0.1000,0.1750,115.005,audio_features,2uMZfydlYpgdYdq76UAQ7A,spotify:track:2uMZfydlYpgdYdq76UAQ7A,https://api.spotify.com/v1/tracks/2uMZfydlYpgd...,https://api.spotify.com/v1/audio-analysis/2uMZ...,257408,4


In [236]:
all_df_merge

,song_name,song_ID,song_artist,Genre,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie,0.714,0.800,11,-4.808,0,0.0504,...,0.3590,0.5890,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie,0.640,0.755,1,-4.077,0,0.0287,...,0.1200,0.7440,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie,0.355,0.918,1,-4.360,1,0.0746,...,0.0971,0.2400,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie,0.462,0.836,11,-5.766,0,0.0621,...,0.1800,0.6710,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby,Indie,0.808,0.381,1,-9.758,1,0.0584,...,0.1120,0.6870,102.052,audio_features,1H6A86dQ7BMdbxofxpQvcK,spotify:track:1H6A86dQ7BMdbxofxpQvcK,https://api.spotify.com/v1/tracks/1H6A86dQ7BMd...,https://api.spotify.com/v1/audio-analysis/1H6A...,174118,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn,Techno,0.751,0.739,5,-8.557,0,0.0380,...,0.0902,0.0988,123.023,audio_features,3KaDPp6hfbnL4dAo57PNCI,spotify:track:3KaDPp6hfbnL4dAo57PNCI,https://api.spotify.com/v1/tracks/3KaDPp6hfbnL...,https://api.spotify.com/v1/audio-analysis/3KaD...,255630,4
3807,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation,Techno,0.619,0.746,7,-10.770,1,0.0407,...,0.1370,0.1700,122.005,audio_features,4nSn1svDL1elP43MZuEc0K,spotify:track:4nSn1svDL1elP43MZuEc0K,https://api.spotify.com/v1/tracks/4nSn1svDL1el...,https://api.spotify.com/v1/audio-analysis/4nSn...,287213,4
3808,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea,Techno,0.781,0.658,1,-10.312,1,0.0446,...,0.1120,0.1060,124.017,audio_features,4M5K6HYMIIoXLutK1v7g7U,spotify:track:4M5K6HYMIIoXLutK1v7g7U,https://api.spotify.com/v1/tracks/4M5K6HYMIIoX...,https://api.spotify.com/v1/audio-analysis/4M5K...,418065,4
3809,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage,Techno,0.792,0.932,10,-6.781,1,0.0500,...,0.0723,0.3560,120.997,audio_features,0hUsx4d1Ur32PPQS9CX7yU,spotify:track:0hUsx4d1Ur32PPQS9CX7yU,https://api.spotify.com/v1/tracks/0hUsx4d1Ur32...,https://api.spotify.com/v1/audio-analysis/0hUs...,388696,4


In [212]:
# just to test in scaling so if goes wrong dont miss the original one

all_df3=all_df

In [213]:
all_df3

,song_name,song_ID,song_artist,Genre,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie,0.714,0.800,11,-4.808,0,0.0504,...,0.3590,0.5890,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie,0.640,0.755,1,-4.077,0,0.0287,...,0.1200,0.7440,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie,0.355,0.918,1,-4.360,1,0.0746,...,0.0971,0.2400,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie,0.462,0.836,11,-5.766,0,0.0621,...,0.1800,0.6710,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby,Indie,0.808,0.381,1,-9.758,1,0.0584,...,0.1120,0.6870,102.052,audio_features,1H6A86dQ7BMdbxofxpQvcK,spotify:track:1H6A86dQ7BMdbxofxpQvcK,https://api.spotify.com/v1/tracks/1H6A86dQ7BMd...,https://api.spotify.com/v1/audio-analysis/1H6A...,174118,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn,Techno,0.751,0.739,5,-8.557,0,0.0380,...,0.0902,0.0988,123.023,audio_features,3KaDPp6hfbnL4dAo57PNCI,spotify:track:3KaDPp6hfbnL4dAo57PNCI,https://api.spotify.com/v1/tracks/3KaDPp6hfbnL...,https://api.spotify.com/v1/audio-analysis/3KaD...,255630,4
3807,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation,Techno,0.619,0.746,7,-10.770,1,0.0407,...,0.1370,0.1700,122.005,audio_features,4nSn1svDL1elP43MZuEc0K,spotify:track:4nSn1svDL1elP43MZuEc0K,https://api.spotify.com/v1/tracks/4nSn1svDL1el...,https://api.spotify.com/v1/audio-analysis/4nSn...,287213,4
3808,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea,Techno,0.781,0.658,1,-10.312,1,0.0446,...,0.1120,0.1060,124.017,audio_features,4M5K6HYMIIoXLutK1v7g7U,spotify:track:4M5K6HYMIIoXLutK1v7g7U,https://api.spotify.com/v1/tracks/4M5K6HYMIIoX...,https://api.spotify.com/v1/audio-analysis/4M5K...,418065,4
3809,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage,Techno,0.792,0.932,10,-6.781,1,0.0500,...,0.0723,0.3560,120.997,audio_features,0hUsx4d1Ur32PPQS9CX7yU,spotify:track:0hUsx4d1Ur32PPQS9CX7yU,https://api.spotify.com/v1/tracks/0hUsx4d1Ur32...,https://api.spotify.com/v1/audio-analysis/0hUs...,388696,4


In [214]:
song_name song_artist Genre

SyntaxError: invalid syntax (<ipython-input-214-3c0370b0f922>, line 1)

### Cleaning all_df3

In [215]:
# deleting the categorical colummns instead of song ID
all_df_4 = all_df3.drop(['type','id', 'uri', 'track_href',
                             'analysis_url'], axis=1)

In [216]:
all_df_4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3811 entries, 0 to 3810
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         3811 non-null   object 
 1   song_ID           3811 non-null   object 
 2   song_artist       3811 non-null   object 
 3   Genre             3811 non-null   object 
 4   danceability      3811 non-null   float64
 5   energy            3811 non-null   float64
 6   key               3811 non-null   int64  
 7   loudness          3811 non-null   float64
 8   mode              3811 non-null   int64  
 9   speechiness       3811 non-null   float64
 10  acousticness      3811 non-null   float64
 11  instrumentalness  3811 non-null   float64
 12  liveness          3811 non-null   float64
 13  valence           3811 non-null   float64
 14  tempo             3811 non-null   float64
 15  duration_ms       3811 non-null   int64  
 16  time_signature    3811 non-null   int64  


In [221]:
# song_ID moves to the index

all_df_4.index=all_df_4.iloc[:,0]

In [222]:
#delete header name for index
all_df_4.rename_axis(None,inplace=True)

In [223]:
all_df_4

,song_name,song_ID,song_artist,Genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
Beggin',Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie,0.714,0.800,11,-4.808,0,0.0504,0.12700,0.000000,0.3590,0.5890,134.002,211560,4
The Less I Know The Better,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie,0.640,0.755,1,-4.077,0,0.0287,0.01380,0.020800,0.1200,0.7440,116.883,216319,4
Mr. Brightside,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie,0.355,0.918,1,-4.360,1,0.0746,0.00119,0.000000,0.0971,0.2400,148.114,222200,4
Boys Don't Cry,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie,0.462,0.836,11,-5.766,0,0.0621,0.00517,0.000000,0.1800,0.6710,168.766,155973,4
wake me up,wake me up,1H6A86dQ7BMdbxofxpQvcK,Ruby,Indie,0.808,0.381,1,-9.758,1,0.0584,0.34400,0.000003,0.1120,0.6870,102.052,174118,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Martian,Martian,3KaDPp6hfbnL4dAo57PNCI,Colyn,Techno,0.751,0.739,5,-8.557,0,0.0380,0.01450,0.884000,0.0902,0.0988,123.023,255630,4
Mandate - Original Mix,Mandate - Original Mix,4nSn1svDL1elP43MZuEc0K,D-Formation,Techno,0.619,0.746,7,-10.770,1,0.0407,0.11600,0.880000,0.1370,0.1700,122.005,287213,4
Hunting - Senses of Mind Remix,Hunting - Senses of Mind Remix,4M5K6HYMIIoXLutK1v7g7U,Mark Tarmonea,Techno,0.781,0.658,1,-10.312,1,0.0446,0.03060,0.742000,0.1120,0.1060,124.017,418065,4
Evolving Flow,Evolving Flow,0hUsx4d1Ur32PPQS9CX7yU,Anturage,Techno,0.792,0.932,10,-6.781,1,0.0500,0.00117,0.916000,0.0723,0.3560,120.997,388696,4


# CLUSTERING TRY 1

In [217]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly 
import plotly.graph_objs as go
from sklearn.cluster import KMeans 
from sklearn import datasets 

In [218]:
kmeans=KMeans(n_clusters=40)
#have to set a starting cluster number 

In [219]:
cluster=kmeans.fit(all_df_5)

NameError: name 'all_df_5' is not defined

In [ ]:
# explore the cluster
cluster.cluster_centers_

In [ ]:
cluster.labels_

In [220]:
#bring cluster into data frame
all_df_5_c=all_df_5.copy()
all_df_5_c['cluster']=cluster.labels_

NameError: name 'all_df_5' is not defined

In [ ]:
all_df_5_c.head()

# SCALING

In [ ]:
from sklearn.preprocessing import StandardScaler 

In [ ]:
all_df_5_scaled=StandardScaler().fit_transform(all_df_5)

In [ ]:
all_df_5_scaled.shape

In [ ]:
all_df_5_scaled_df=pd.DataFrame(all_df_5_scaled,columns=['danceability','energy','key',
                                                         'loudness', 'mode', 'speechiness', 'acousticness', 
                                                         'instrumentalness', 'liveness', 
                                                         'valence', 'tempo', 'duration_ms',
                                                         'time_signature' ])

In [ ]:
all_df_5_scaled_df.head()

In [ ]:
all_df_5_scaled_df.describe()

In [ ]:
all_df_5_scaled_df.hist(bins=15, figsize=(15,15));

# Finding the optimal K

In [ ]:
kmeans.inertia_

In [237]:
kmeans=KMeans(n_clusters=5)
kmeans.fit(all_df_5)
print(kmeans.inertia_)

NameError: name 'all_df_5' is not defined

In [ ]:
K= range(16,50)

In [ ]:
inertia = []
for k in K:
    kmeans=KMeans(n_clusters=k,random_state=40)
    kmeans.fit(all_df_5)
    inertia.append(kmeans.inertia_)
    
print(inertia)

In [ ]:
# plot the results

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K),max(K)+1,1.0))
plt.title('elbow method, optimal k by inertia');

# CLUSTERING TRY 2

In [ ]:
kmeans=KMeans(n_clusters=31)
#have to set a starting cluster number 

In [ ]:
cluster=kmeans.fit(all_df_5)

In [ ]:
# explore the cluster
cluster.cluster_centers_

In [ ]:
cluster.labels_

In [ ]:
#bring cluster into data frame
all_df_5_copy=all_df_5.copy()
all_df_5_copy['cluster']=cluster.labels_

In [ ]:
all_df_5_copy.head()

In [ ]:
all_df_5_copy

### Visually explore the clusters

In [ ]:
import seaborn as sns
sns.scatterplot(data=all_df_5, x="danceability", y="tempo",hue='cluster')

### 3-D plotting with plotly 

In [ ]:
markersize= all_df_5['tempo']
markercolour= all_df_5['cluster']

fig1=go.Scatter3d(x=all_df_5['danceability'],
                  y=all_df_5['energy'],
                  z=all_df_5['acousticness'],
                 marker=dict (size=markersize,
                              color=markercolour,
                              opacity=0.9,
                              colorscale='Blues'),
                 line=dict (width=0.2),
                 mode='markers')
#make layout 
mylayout=go.Layout(scene=dict(xaxis=dict(title="danceability"),
                              yaxis=dict(title='energy'),
                              zaxis=dict(title='acousticness')))

In [ ]:
plotly.offline.plot({"data":[fig1],
                    "layout":mylayout},
                   auto_open=True,
                   filename=("5Dplot.html"))

# Getting the final DF with the cluster and specifications

In [ ]:
all_df_5_copy

### I put into "Raw NBConvert" some lines to make the notebook easily to read